In [35]:
%%writefile ../src/BankV3.sol
pragma solidity ^0.6.0;

contract BankV3 {
    address owner;
    uint balance;
    uint256 timeToDeposit;
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }

    function forwardTo(address payable _receiver) public payable onlyOwner {
        //require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit = now + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        //require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    modifier onlyAfter {
        require(now >= timeToDeposit);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Overwriting ../src/BankV3.sol


In [36]:
!solc ../src/BankV3.sol --combined-json abi > ..\src\BankV3ABI.json

In [37]:
!solc ../src/BankV3.sol --combined-json bin > ..\src\BankV3BIN.json

In [38]:
%%writefile ../src/BankV3Deploy.js
var Web3=require('web3');
var _abiJson = require('./BankV3ABI.json');
var _binJson = require('./BankV3BIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);
_bin=_binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/BankV3Deploy.js


In [39]:
!node ../src/BankV3Deploy.js

- contract name:  ../src/BankV3.sol:BankV3
Deploying the contract from 0x6d21d579193C480f321F69c581A2a91E41BAe32D
hash: 0x3d4e327c65f05b671917d9b8b380955597218550d21715321ef4be6d170a9551
---> The contract deployed to: 0xeFbcE6949851E5c4F68DE5027006c144852a898C


In [44]:
%%writefile ../src/BankV3Use.js
var Web3=require('web3');
var _abiJson = require('./BankV3ABI.json');
var _binJson = require('./BankV3BIN.json');


var web3;
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _bank = new web3.eth.Contract(_abiArray, '0xeFbcE6949851E5c4F68DE5027006c144852a898C');

async function doIt() {
    const accounts = await web3.eth.getAccounts();

    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    _bank.methods.getBalance().call().then(console.log);
    
    await _bank.methods.deposit(111).send({from: accounts[0], value:111});
    _bank.methods.getBalance().call().then(console.log);
    
    await _bank.methods.withdrawAll().send({from: accounts[0]});    //greater than 101
    _bank.methods.getBalance().call().then(console.log);
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    

    
}
doIt()

Overwriting ../src/BankV3Use.js


In [45]:
!node ../src/BankV3Use.js

- contract name:  ../src/BankV3.sol:BankV3
Balance before: 99998930688000000000
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Result { '0': '0', '1': '0' }
Balance after: 99997258368000000000
Balance diff: 1672320000000000
